In [ ]:
# Installe la bibliothèque Ultralytics
!pip install ultralytics

# Importe la fonction 'clear_output' du module 'display' pour effacer la sortie précédente
from IPython import display
display.clear_output()

# Vérifie les dépendances Ultralytics
import ultralytics
ultralytics.checks()

# Importe le module 'os' pour travailler avec le système de fichiers
import os

# Obtient le répertoire de travail actuel
HOME = os.getcwd()

# Se déplace vers le répertoire de travail actuel
%cd {HOME}

# Clone le référentiel GitHub ByteTrack dans le répertoire actuel
!git clone https://github.com/ifzhang/ByteTrack.git

# Se déplace vers le répertoire ByteTrack
%cd {HOME}/ByteTrack

# Correction liée à un problème dans requirements.txt
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

# Installe les dépendances à partir de requirements.txt
!pip3 install -q -r requirements.txt

# Installe le package ByteTrack en mode développement
!python3 setup.py -q develop

# Installe les packages cython_bbox et onemetric
!pip install -q cython_bbox
!pip install -q onemetric

# Correction liée à des problèmes dans le notebook
!pip install -q loguru lap thop

# Efface la sortie précédente
from IPython import display
display.clear_output()

# Ajoute le chemin vers le répertoire ByteTrack au chemin d'importation
import sys
sys.path.append(f"{HOME}/ByteTrack")

# Importe le module yolox et affiche sa version
import yolox
print("yolox.__version__:", yolox.__version__)

# Installe la version spécifique du package supervision
!pip install supervision==0.1.0
display.clear_output()

# Importe le module supervision et affiche sa version
import supervision
print("supervision.__version__:", supervision.__version__)

# Importe les modules nécessaires
from supervision.tools.detections import Detections, BoxAnnotator
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx
import pandas as pd
from PIL import Image
import csv
import math
from ultralytics import YOLO
import time
from keras.models import load_model
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from typing import List
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass
from tqdm.notebook import tqdm

supervision.__version__: 0.1.0


In [10]:
# Décorateur @dataclass pour définir une classe de configuration BYTETrackerArgs
@dataclass(frozen=True)
class BYTETrackerArgs:
    # Seuil pour décider si une piste est activée ou non
    track_thresh: float = 0.4

    # Taille du tampon de piste (nombre maximal de trames de suivi)
    track_buffer: int = 300

    # Seuil de correspondance pour associer les détections aux pistes
    match_thresh: float = 0.8

    # Seuil de rapport d'aspect pour filtrer les boîtes englobantes basées sur leur forme
    aspect_ratio_thresh: float = 3.0

    # Surface minimale d'une boîte englobante pour être prise en compte
    min_box_area: float = 1.0

    # Indicateur pour activer/désactiver des paramètres spécifiques à mot20
    mot20: bool = False


In [ ]:
# Cette fonction convertit les détections en un format pouvant être utilisé par la fonction match_detections_with_tracks
def detections2boxes(detections: Detections) -> np.ndarray:
    # Concatène les coordonnées xyxy des détections avec les confiances correspondantes
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# Cette fonction convertit une liste de STrack en un format pouvant être utilisé par la fonction match_detections_with_tracks
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    # Crée un tableau numpy contenant les boîtes (tlbr) des pistes
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# Cette fonction associe nos boîtes englobantes avec les prédictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    # Vérifie si les coordonnées xyxy des détections sont vides ou s'il n'y a pas de pistes
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    # Convertit les boîtes des pistes en un tableau numpy
    tracks_boxes = tracks2boxes(tracks=tracks)

    # Calcule l'indice de recouvrement IoU (Intersection over Union) entre les boîtes des pistes et les détections
    iou = box_iou_batch(tracks_boxes, detections.xyxy)

    # Pour chaque détection, trouve l'indice de la piste avec lequel elle a le plus grand IoU
    track2detection = np.argmax(iou, axis=1)

    # Initialise une liste vide pour stocker les ID des pistes associées aux détections
    tracker_ids = [None] * len(detections)

    # Associe les ID des pistes aux détections en fonction des indices d'association
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    # Retourne la liste des ID de pistes associées aux détections
    return tracker_ids


In [11]:
# Chemin d'accès à la vidéo source
SOURCE_VIDEO_PATH = f"/content/nvcxWZBO.mp4"
# Chemin d'accès à la vidéo cible (résultante)
TARGET_VIDEO_PATH = f"/content/result.mp4"

# Configuration
MODEL = "yolov8x.pt"
model = YOLO(MODEL)  # Crée une instance du modèle YOLO
CLASS_NAMES_DICT = model.model.names  # Dictionnaire qui mappe les IDs de classe aux noms de classe
CLASS_ID = [0]  # Liste d'IDs de classe d'intérêt (par exemple, ID de la classe "voiture")
byte_tracker = BYTETracker(BYTETrackerArgs())  # Crée une instance de BYTETracker avec les arguments spécifiés
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)  # Obtient des informations sur la vidéo source
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)  # Génère des images de la vidéo source
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=2, text_thickness=2, text_scale=1)  # Crée une instance de BoxAnnotator pour annoter les boîtes

# Ouvre le fichier vidéo cible pour écriture
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # Parcours des images de la vidéo en boucle
    for frame in tqdm(generator, total=video_info.total_frames):
        # Prédiction du modèle sur une image individuelle et conversion en détections de supervision
        results = model(frame, classes=[0])
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # Filtrage des détections avec des classes indésirables
        massk = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections = detections.filter(mask=massk, inplace=True)
        # Suivi des détections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # Filtrage des détections sans suivi
        maskk = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=maskk, inplace=True)
        # Formatage des libellés personnalisés
        labels = [
            f"ID_{tracker_id}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        # Annotation et affichage de l'image
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        sink.write_frame(frame)  # Écrit l'image annotée dans la vidéo cible


  0%|          | 0/141 [00:00<?, ?it/s]


0: 384x640 (no detections), 64.0ms
Speed: 2.2ms preprocess, 64.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.7ms
Speed: 2.1ms preprocess, 44.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.3ms
Speed: 2.0ms preprocess, 39.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.5ms
Speed: 2.1ms preprocess, 39.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.9ms
Speed: 2.0ms preprocess, 35.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.5ms
Speed: 1.9ms preprocess, 35.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.0ms
Speed: 1.8ms preprocess, 31.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.7ms
Speed: 2.1ms preprocess, 30.7ms inferenc